In [1]:
import pandas as pd 
import numpy as np 
import jieba
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from collections import defaultdict

'''实现步骤：
1.加载数据集
2.对数据进行处理，将数据中['content']列为空的值清除
3.加载停用词表
4.对数据集中的每条数据进行处理，清除空格和换行符，并使用jieba.cut（）函数对数据集中内容进行分词。默认采用精确模式。
5.对照停用词表将分词后的词语进行删除，减少不必要的数据。
6.采用TFIDF方法，进行文本的特征处理
7.进行训练集和样本集的数据切分；训练内容是词频数据，标签是1或0,1代表'新华社'，0代表‘非新华社’。词频特征体现了文章的风格。
8.采用多项式贝叶斯分类器进行训练，获得分类模型
9.利用模型对文章进行预测
10.使用K-means方法将数据集内的文章分为25类。
11.将新华社文章索引统计出来;
12.输入需要查询的文章索引，在统计出的新华社文章索引集内进行查找
13.若涉嫌抄袭，将查询到的文章显示出来，top=10,只显示前10篇。
'''

pd.options.display.max_columns = None
pd.options.display.max_rows = None
#数据加载
news=pd.read_csv('sqlResult.csv', encoding='gb18030')
#print(news.shape)
#print(news.head())

#print(news[news.content.isna()].head())
news=news.dropna(subset=['content'])
#print(news)

#加载停用词表
with open('chinese_stopwords.txt','r',encoding='utf-8') as file:
	stopwords=[i[:-1] for i in file.readlines()]

def split_text(text):
	text=text.replace(' ','').replace('\n','')
	text2=jieba.cut(text)
	result=' '.join(w for w in text2 if w not in stopwords)
	return result

corpus=list(map(split_text,[str(i) for i in news.content]))
print(corpus)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.828 seconds.
Prefix dict has been built successfully.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
#print(corpus)
#计算corpus的TFIDF
countvectorizer=CountVectorizer(encoding='gb18030',min_df=0.015)  #将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i文本下的词频。当词频小于0.015时，该词不作为关键词。
tfidftransformer=TfidfTransformer()#统计每个词语的tf-idf权值
print(tfidftransformer)
countvector=countvectorizer.fit_transform(corpus)#将corpus文本转为词频矩阵
print(countvector)
tfidf=tfidftransformer.fit_transform(countvector)#计算tf-idf
print(tfidf)


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
  (0, 590)	1
  (0, 461)	1
  (0, 271)	2
  (0, 416)	1
  (0, 432)	1
  (0, 669)	1
  (0, 860)	1
  (0, 184)	1
  (0, 822)	1
  (0, 385)	1
  (0, 103)	1
  (0, 667)	1
  (0, 263)	1
  (0, 44)	1
  (0, 174)	1
  (1, 399)	3
  (1, 424)	1
  (1, 67)	1
  (1, 721)	1
  (1, 579)	1
  (1, 468)	1
  (1, 50)	1
  (1, 525)	2
  (1, 474)	1
  (1, 391)	1
  :	:
  (87053, 817)	4
  (87053, 598)	11
  (87053, 571)	1
  (87053, 641)	1
  (87053, 684)	1
  (87053, 42)	1
  (87053, 294)	1
  (87053, 58)	1
  (87053, 533)	1
  (87053, 649)	1
  (87053, 46)	1
  (87053, 405)	1
  (87053, 81)	1
  (87053, 527)	1
  (87053, 106)	1
  (87053, 351)	1
  (87053, 594)	1
  (87053, 208)	2
  (87053, 437)	7
  (87053, 869)	1
  (87053, 882)	6
  (87053, 249)	1
  (87053, 342)	1
  (87053, 110)	3
  (87053, 851)	1
  (0, 860)	0.26741301446611326
  (0, 822)	0.15081854599522596
  (0, 669)	0.26715029976903004
  (0, 667)	0.28310913637357893
  (0, 590)	0.2377366084883309
  (0, 461)	0.2878

In [9]:
print(tfidf.shape)

(87054, 884)


In [3]:
#标记是否为自己的新闻
label=list(map(lambda source:1 if '新华社' in str(source) else 0,news.source))
#print(label)

#对数据集进行切分
X_train,X_test,y_train,y_test=train_test_split(tfidf.toarray(),label,test_size=0.3) #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

model=MultinomialNB()#多项式贝叶斯分类
model.fit(X_train,y_train)

y_predict=model.predict(X_test)

#使用模型进行预测
prediction=model.predict(tfidf.toarray())
labels=np.array(label)
#compare_news_index显示两列，prediction为预测值，labels为真实结果
compare_news_index=pd.DataFrame({'prediction':prediction,'labels':labels})
#涉嫌抄袭的文章
copy_news_index=compare_news_index[(compare_news_index['prediction']==1)&(compare_news_index['labels']==0)].index 
#实际为新华社的文章
xinhuashe_news_index=compare_news_index[(compare_news_index['prediction']==1)&(compare_news_index['labels']==1)].index
print(xinhuashe_news_index)


Int64Index([    3,    39,   310,   543,   649,   669,   677,   729,   736,
              907,
            ...
            87042, 87043, 87044, 87045, 87046, 87047, 87048, 87049, 87051,
            87053],
           dtype='int64', length=71351)


In [4]:
print(copy_news_index)

Int64Index([   4,   24,   25,   28,   30,   32,   35,   37,   38,   43,
            ...
            8535, 8538, 8539, 8543, 8546, 8548, 8549, 8551, 8552, 8555],
           dtype='int64', length=2781)


In [5]:
#使用Kmeans进行全量文档的聚类
normalizer=Normalizer()
scaled_array=normalizer.fit_transform(tfidf.toarray())
kmeans=KMeans(n_clusters=25)
k_labels=kmeans.fit_predict(scaled_array)
print(k_labels)

id_class={index:class_ for index,class_ in enumerate(k_labels)}
print(id_class)

[ 8  8  8 ... 22 21  8]
{0: 8, 1: 8, 2: 8, 3: 6, 4: 8, 5: 3, 6: 8, 7: 3, 8: 8, 9: 3, 10: 3, 11: 3, 12: 8, 13: 3, 14: 3, 15: 3, 16: 8, 17: 8, 18: 3, 19: 3, 20: 22, 21: 3, 22: 8, 23: 3, 24: 8, 25: 8, 26: 8, 27: 8, 28: 8, 29: 9, 30: 8, 31: 8, 32: 9, 33: 8, 34: 8, 35: 8, 36: 8, 37: 8, 38: 8, 39: 6, 40: 8, 41: 8, 42: 9, 43: 8, 44: 8, 45: 16, 46: 3, 47: 3, 48: 8, 49: 8, 50: 8, 51: 6, 52: 9, 53: 8, 54: 8, 55: 8, 56: 8, 57: 9, 58: 9, 59: 9, 60: 8, 61: 8, 62: 9, 63: 20, 64: 16, 65: 9, 66: 16, 67: 9, 68: 9, 69: 8, 70: 8, 71: 16, 72: 9, 73: 8, 74: 8, 75: 8, 76: 8, 77: 8, 78: 8, 79: 8, 80: 8, 81: 8, 82: 8, 83: 8, 84: 8, 85: 8, 86: 9, 87: 8, 88: 16, 89: 9, 90: 9, 91: 8, 92: 8, 93: 8, 94: 9, 95: 9, 96: 16, 97: 8, 98: 9, 99: 9, 100: 3, 101: 8, 102: 8, 103: 8, 104: 8, 105: 3, 106: 9, 107: 8, 108: 8, 109: 9, 110: 3, 111: 9, 112: 9, 113: 9, 114: 8, 115: 3, 116: 9, 117: 9, 118: 9, 119: 9, 120: 8, 121: 16, 122: 8, 123: 11, 124: 8, 125: 9, 126: 8, 127: 21, 128: 16, 129: 9, 130: 3, 131: 8, 132: 8, 133: 11, 

In [19]:
#定义class_id用于统计新华社发布的文章的id.
class_id=defaultdict(set)
for index,class_ in id_class.items():
	if index in xinhuashe_news_index.tolist():
		class_id[class_].add(index)
print(class_id)

defaultdict(<class 'set'>, {6: {3, 32785, 32786, 65561, 65566, 65572, 65573, 65574, 39, 65575, 65587, 65588, 65589, 65591, 65593, 65619, 32853, 32854, 32855, 32856, 32857, 32858, 32866, 65635, 65636, 65637, 32870, 32871, 32872, 32873, 32874, 32875, 32876, 32877, 32878, 32879, 32880, 32881, 32882, 32883, 32884, 32885, 32886, 32887, 32888, 32889, 32890, 32891, 32892, 32893, 32894, 32895, 65663, 32897, 65665, 65666, 65669, 65670, 65671, 65673, 65674, 65675, 65677, 65696, 65698, 65699, 65700, 65701, 65702, 65706, 32951, 65726, 65727, 32966, 32968, 32995, 32996, 32997, 33002, 33003, 33004, 33005, 33006, 33007, 33008, 65771, 33022, 33023, 33026, 33028, 33029, 33097, 33098, 33099, 65936, 65946, 65947, 65948, 33203, 33204, 33209, 33210, 33211, 33225, 33226, 33227, 33228, 66018, 66019, 66020, 66031, 66041, 33283, 33284, 66070, 66071, 66072, 66078, 66079, 66080, 66081, 66128, 66129, 66130, 66131, 66132, 66133, 66136, 33369, 33370, 33371, 33372, 66137, 66146, 66147, 66148, 66150, 66157, 66169, 33

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
#查找相似的文章
def find_similar_text(cpindex,top=10):
	dis_dict={i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
	return sorted(dis_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

cpindex=int(input('请输入需要查询的文章索引：'))

print('是否在新华社:',cpindex in xinhuashe_news_index)
print('是否存在抄袭：',cpindex in copy_news_index)

请输入需要查询的文章索引：4
是否在新华社: False
是否存在抄袭： True


In [30]:
similar_list=find_similar_text(cpindex)
print(similar_list)

print('怀疑抄袭:\n',news.iloc[cpindex].content)
#找到的一篇相似的文章
similar2=similar_list[0][0]
print('相似的原文：\n',news.iloc[similar2].content)

[(50852, array([[0.85814188]])), (50720, array([[0.85215671]])), (67073, array([[0.73866006]])), (12505, array([[0.70321003]])), (40457, array([[0.6966889]])), (9008, array([[0.63271285]])), (13155, array([[0.63062198]])), (73984, array([[0.61087536]])), (48190, array([[0.60472756]])), (46602, array([[0.56308961]]))]
怀疑抄袭:
 （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）
@深圳交警微博称：昨日清晨交警发现有一女子赤裸上身，行走在南坪快速上，期间还起了轻生年头，一辅警发现后赶紧为其披上黄衣，并一路劝说她。
那么事发时
到底都发生了些什么呢？
南都记者带您一起还原现场
南都记者在龙岗大队坂田中队见到了辅警刘青（发现女生的辅警），一位外表高大帅气，说话略带些腼腆的90后青年。
刘青介绍，6月16日早上7时36分，他正在环城南路附近值勤，接到中队关于一位女子裸身进入机动车可能有危险的警情，随后骑着小铁骑开始沿路寻找，大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子。
女子身上一丝不挂地逆车流而行，时走时停，时坐时躺，险象环生。刘青停好小铁骑，和另外一名巡防员追了上去，发现女子的情绪很低落，话不多，刘青尝试和女子交流，劝说女子离开，可女子并不愿意接受，继续缓慢地往南坪快速路的主干道上走去。
此时路边上已经聚集了很市民围观，为了不刺激女子的情绪，刘青和巡防员一边盯着女子一边驱赶着围观的群众。
现场还原
从警方提供的一份视频了解到，16日早上7时25分，女子出现在坂雪岗大道与环城南路的监控视频中，此时女子还穿着白色的内裤，正沿着坂雪岗大道往南坪快速的方向缓慢地走着。
当时正值上班高峰期，十字路口的车流已经排起了长队。当女子出现时，路上的市民纷纷驻足观望，不少车辆也放慢了速度，但女子并不为市民观望停下脚步，依然缓慢走着。当女子行进到十字路口中间时，一辆大货车挡住了镜头，但是当女子再次出现镜头时，可以发现女子已经没穿内裤了，全身裸露